# Game and Argumentation: Time for a Family Reunion

This notebook aims to demonstrate the paper "Game and Argumentation: Time for a Family Reunion" accepted by XLoKR 2023

The move graph above defines a game $G = (V, E)$ with positions $V$ and moves $E$. Based on the $win(X) ← move(X, Y ), ¬ win(Y ).$, we can solve this game by
1) Identifying sink nodes, or lost positions which did not have outgoing moves
2) Then we iteratively run the following rules:
- Position $x$ is won (green) if $∃ y$ s.t. $move(x,y)$ and position $y$ is already lost (red)
- Position $x$ is lost (red) if $∀ y$ if $move(x,y)$ then position $y$ is already won (green)

## Mode Selection

In [1]:
mode = "game"
# mode = "argumentation"

## Move Graph

To help future reasoning, here we turn the graphs into facts

In [2]:
%%file files/move_graph.dlv

#maxint = 100.

m(a,b).
m(a,c).
m(k,l).
m(c,d).
m(c,e).
m(l,e).
m(l,m).
m(d,e).
m(e,d).
m(e,m).
m(m,e).
m(d,f).
m(d,g).
m(g,d).
m(e,h).
m(m,n).
m(n,m).

Overwriting files/move_graph.dlv


#### Extract Move Graph Structure

In [3]:
import re
import pandas as pd

def read_edges_from_file(filename):
    edges = []
    with open(filename, 'r') as file:
        for line in file:
            # Extract edges that match the pattern "m(source, target)"
            match = re.match(r'm\(([^,]+),([^)]+)\)', line)
            if match:
                source = match.group(1)
                target = match.group(2)
                edges.append((source, target))
    return edges

if __name__ == "__main__":
    file_name = "files/move_graph.dlv"
    edges = read_edges_from_file(file_name)
    edge_df = pd.DataFrame(edges, columns=['source', 'target'])

In [4]:
edge_df

,source,target
0,a,b
1,a,c
2,k,l
3,c,d
4,c,e
5,l,e
6,l,m
7,d,e
8,e,d
9,e,m


# Solve Game

### Game Solve Script (DLV)

In [5]:
%%file files/game_solve.dlv

% Positions
p(X) :- m(X,_).
p(X) :- m(_,X).

% win_u: underestimate of WON positions
u(S1, X) :-
	m(X,Y),
	not o(S,Y),
	nxt(S,S1). % S1 = S + 1

%: win_o: overestimate of WON positions
o(S, X) :-
	m(X,Y),
	not u(S,Y),
	nxt(S,_).
% GREEN (won) positions 
g(X) :-
	fg(_,X).  

% YELLOW (drawn) positions
y(X) :-
	p(X),
	not g(X),
	not r(X).

% RED (lost) positions
r(X) :- fr(_,X).

% State generation for AFP 
nxt(0,1).
nxt(S,S1) :-			% S1 (=S+1) is a state,
	nxt(_,S),		% ... if S is a state
	chg(S),			% ... which changes
	S1=S+1.    

% change(S)
chg(0).				% in 0 there is change
chg(S) :-			% in S there is change
	fg(S,_).		% ... if there is some FirstGreen

% final(S)
fin(S) :-			% S is the final state
	nxt(_,S),     
	not chg(S).		% ... if there is no change in S

% FirstGreen(State, Position)
fg(S1,X) :- 		       % position X is first green in S1 (=S+1)
	nxt(S,S1),
	u(S1,X),               % ... if win_u(S1,X)
	not u(S,X).            % ... but not win_u(S,X)

% FirstRed(State, Position)
fr(0,X) :-                     % X is first red in 0
	p(X),                  % ... if X is a position
	not o(0,X).            % ... that isn't even in the first overestimate (at 0)

fr(S1,X) :-                    % X is first red in S1 (=S+1)
	nxt(S,S1),
	o(S,X),                % ... if X was in the previous overestimate win_o(S,X)
	not o(S1,X),           % ... but isn't now in win_o(S1,X)
	not fin(S1).           % but exclude final state (we don't compute o(Final,...) )
%	not u(S1,X).           




% node(Color, State, Position)
node(g,S,X) :- fg(S,X).
node(r,S,X) :- fr(S,X).
node(y,S,X) :- y(X),fin(S).


outn(gr,S1,X,Y) :-
	m(X,Y), nxt(S,S1),
	g(X), fr(S,Y).		% GREEN --(s+1)--> FIRST-RED(s) 
outn(rg,S,X,Y) :- m(X,Y), r(X), fg(S,Y).

out(gg,X,Y) :- m(X,Y), g(X), g(Y).
out(gy,X,Y) :- m(X,Y), g(X), y(Y).
out(yg,X,Y) :- m(X,Y), y(X), g(Y).
% out(x,X,Y) :- m(X,Y), r(X), r(Y).
% out(x,X,Y) :- m(X,Y), r(X), y(Y).
% out(x,X,Y) :- m(X,Y), y(X), r(Y).
out(yy,X,Y) :- m(X,Y), y(X), y(Y).

Overwriting files/game_solve.dlv


In [6]:
!dlv files/move_graph.dlv files/game_solve.dlv -filter="node"

DLV [build BEN/Dec 17 2012   gcc 4.6.1]

{node(r,0,b), node(r,0,f), node(r,0,h), node(g,1,a), node(g,1,d), node(g,1,e), node(r,1,c), node(r,1,g), node(y,2,k), node(y,2,l), node(y,2,m), node(y,2,n)}


### Process DLV Output

For a clear demonstration, we parse the output of DLV into a DataFrame

In [7]:
import pandas as pd
import subprocess
import re

def run_command(cmd):
    result = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
    output = result.stdout.decode()
    return output

def to_dataframe(output):
    output = output[1:-1]

    # Split the output into separate items
    items = output.split(", ")

    # Prepare the data for DataFrame
    data = []
    for item in items:
        # Extract the elements
        elements = re.findall(r'node\((.*?),(.*?),(.*?)\)', item)[0]
        data.append(elements)

    # Create DataFrame
    df = pd.DataFrame(data, columns=['color', 'state_id', 'node_label'])
    
    df=df[['state_id', 'node_label', 'color']]
    return df

if __name__ == "__main__":
    cmd = "dlv files/move_graph.dlv files/game_solve.dlv -filter='node',"
    output = run_command(cmd)
    state_df = to_dataframe(output)

In [8]:
state_df

,state_id,node_label,color
0,0,b,r
1,0,f,r
2,0,h,r
3,1,a,g
4,1,d,g
5,1,e,g
6,1,c,r
7,1,g,r
8,2,k,y
9,2,l,y


### State ID Redefine

In [9]:
# Count unique colors for each state_id
color_counts = state_df.groupby('state_id')['color'].nunique()

# Create a mapping for colors to their respective increments
color_mapping = {'g': .1, 'r': .2}  # Adjust this mapping based on your requirement

# Create a new state_id column
state_df['new_state_id'] = state_df['state_id'].astype(float)

# Apply the mapping to the color column only if there are more than one unique color for that state_id
state_df['new_state_id'] = state_df.apply(lambda row: row['new_state_id'] + color_mapping.get(row['color'], 0) 
                                      if color_counts[row['state_id']] > 1 else row['new_state_id'], axis=1)

# Replace the old state_id column with the new one:
state_df['state_id'] = state_df['new_state_id']
state_df.drop(columns=['new_state_id'], inplace=True)
state_df = state_df.sort_values(by='state_id')

In [10]:
state_df

,state_id,node_label,color
0,0.0,b,r
1,0.0,f,r
2,0.0,h,r
3,1.1,a,g
4,1.1,d,g
5,1.1,e,g
6,1.2,c,r
7,1.2,g,r
8,2.0,k,y
9,2.0,l,y


## Color the Node State by State

In [11]:
import pandas as pd
import pygraphviz as pgv
from IPython.display import Image
import os

def delete_directory_contents(dir_path):
    for filename in os.listdir(dir_path):
        file_path = os.path.join(dir_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                delete_directory_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')

directory_path = 'output'
delete_directory_contents(directory_path)

# Stylesheet set up
color_map_game = {'r': '#FFAAAA', 'g': '#AAFFAA', 'y': '#FFFFAA'}
color_map_argumentation = {'r': '#bfefff', 'g': '#ffdaaf', 'y': '#ffffbf'}

edge_dir_game="forward"
edge_dir_argumentation="back"

# Use different stylesheet for game and argumentation
if mode == "game":
    color_map=color_map_game
    edge_dir=edge_dir_game
else:
    color_map=color_map_argumentation
    edge_dir=edge_dir_argumentation

# Create a directed graph from the first dataframe
G = pgv.AGraph(directed=True)

# Adding edges from the first dataframe
for _, row in edge_df.iterrows():
    G.add_edge(row['source'], row['target'], dir=edge_dir)
    G.write(f'output/uncolored_graph.dot')

# For each state, highlight the node and write to a separate dot file
for _, row in state_df.iterrows():
    # Find the node corresponding to the state
    node = G.get_node(row['node_label'])
    node.attr['fillcolor'] = color_map[row['color']]
    node.attr['style'] = 'filled'
    # Export to a Graphviz dot file
    G.write(f'output/state_{row["state_id"]}_graph.dot')

## Color Edges of Move Graph

In [12]:
# Digest the node colored graph
node_colored_graph='output/state_{}_graph.dot'.format(max(state_df["state_id"]))
G = pgv.AGraph(node_colored_graph)

# Get the color for each node
node_color_dict = state_df.set_index('node_label')['color'].to_dict()

# Adding edges from the first dataframe
for _, row in edge_df.iterrows():
    source_node = row['source']
    target_node = row['target']
    source_col = node_color_dict[source_node]
    target_col = node_color_dict[target_node]
    edge = G.get_edge(source_node, target_node)
    # Check the colors and change the edge color accordingly
    if source_col == 'g' and target_col == 'r':
        if mode == "game":
            edge.attr['color'] = '#00BB00' #green
        else:
            edge.attr['color'] = '#006ad1' #blue
    elif source_col == 'r' and target_col == 'g':
        if mode == "game":
            edge.attr['color'] = '#CC0000' #red
        else:
            edge.attr['color'] = '#cc8400' #orange
    elif source_col == 'y' and target_col == 'y':
        edge.attr['color'] = '#AAAA00' #yellow
    else:
        edge.attr['color'] = '#b7b7b7'
        edge.attr['style'] = 'dashed'
        
G.write(f'output/colored_edge_graph.dot')

## dot to PNG

In [13]:
import os

# List all files in the output directory
files = os.listdir('output')

# Filter out the .dot files
dot_files = [file for file in files if file.endswith('.dot')]


for dot_file in dot_files:
    with open(f'output/{dot_file}', 'r') as f:
            lines = f.readlines()

    # Insert the new line before the last line
    # lines.insert(-1, '{rank=same; "d" "e" "m"}\n')  # your custom script

    # Write the modified lines back to the file
    with open(f'output/{dot_file}', 'w') as f:
        f.writelines(lines)
        
    png_file = os.path.splitext(dot_file)[0] + '.png'  # Replace .dot extension with .png
    command = f'dot -Tpng output/{dot_file} -o output/{png_file}'
    os.system(command)

# Animation

In [14]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import re

image_files = os.listdir('output')
# Filter out only the files of interest
state_files = [f for f in image_files if f.startswith('state_') and f.endswith('_graph.png')]
# Sort these files based on the state_id
state_files.sort(key=lambda f: float(re.search(r'(\d+(\.\d+)?)', f).group(1)))
state_files = ['output/' + f for f in state_files]
# Append the special files
image_files = ['output/uncolored_graph.png'] + state_files + ['output/colored_edge_graph.png']

# Create widgets
image_slider = widgets.IntSlider(value=0, min=0, max=len(image_files)-1, step=1, description='Slide:')
prev_button = widgets.Button(description='Previous')
next_button = widgets.Button(description='Next')
output = widgets.Output()

# Define button click events
def on_prev_button_clicked(b):
    if image_slider.value > 0:
        image_slider.value -= 1

def on_next_button_clicked(b):
    if image_slider.value < len(image_files)-1:
        image_slider.value += 1

prev_button.on_click(on_prev_button_clicked)
next_button.on_click(on_next_button_clicked)

# Define slider event
def on_value_change(change):
    with output:
        clear_output(wait=True)
        img = mpimg.imread(image_files[image_slider.value])
        plt.imshow(img)
        plt.axis('off')
        plt.show()

image_slider.observe(on_value_change, names='value')

# Display widgets
display(widgets.HBox([prev_button, next_button]))
display(image_slider)
display(output)

# Initial image
with output:
    img = mpimg.imread(image_files[0])
    plt.imshow(img)
    plt.axis('off')
    plt.show()

IntSlider(value=0, description='Slide:', max=5)

Output()